In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import requests
from json import load
import jsonschema
from jsonschema import validate
import json


# Open JSON file

In [2]:
# schema_file = "Agri_schema.json"
# schema_df = pd.read_json(schema_file)
# schema_df.head()

In [3]:
# with open('Agri_schema.json') as sc:
#     schema = load(sc)

# print(schema)

In [4]:
with open('Agri_schema.json', "r") as sc:
    schema = load(sc)
    
    pretty_json = json.dumps(schema, indent=3)
    print(pretty_json)

{
   "title": "agri-environmental-indicators-emissions-by-sector",
   "type": "object",
   "oneOf": [
      {
         "$ref": "#/definitions/agri-environmental-indicators-emissions-by-sector"
      }
   ],
   "definitions": {
      "agri-environmental-indicators-emissions-by-sector": {
         "properties": {
            "records": {
               "type": "array",
               "items": {
                  "$ref": "#/definitions/agri-environmental-indicators-emissions-by-sector_records"
               }
            }
         }
      },
      "agri-environmental-indicators-emissions-by-sector_records": {
         "properties": {
            "fields": {
               "type": "object",
               "properties": {
                  "year": {
                     "type": "string",
                     "format": "date",
                     "title": "Year",
                     "description": ""
                  },
                  "area": {
                     "type": "string",


In [5]:
json_file ="agri-environmental-indicators-emissions-by-sector.json"

In [6]:
entire_file_df = pd.read_json(json_file)
entire_file_df.head()

,datasetid,recordid,fields,record_timestamp
0,agri-environmental-indicators-emissions-by-sector,5500592b928d56fc0da60501fa364fa5c132b79a,"{'year': '2014', 'element': 'Emissions (F-gase...",2021-11-07T05:29:57.595-06:00
1,agri-environmental-indicators-emissions-by-sector,6b85b6ddf5ef6956602f5794450b16671561f5ff,"{'year': '2018', 'element': 'Emissions (F-gase...",2021-11-07T05:29:57.595-06:00
2,agri-environmental-indicators-emissions-by-sector,7315471f71541c41f2a5349f7bb4b522a051931f,"{'year': '1991', 'element': 'Emissions Share (...",2021-11-07T05:29:57.595-06:00
3,agri-environmental-indicators-emissions-by-sector,825a93245b1d222469a12604a1651cc34a63bd52,"{'year': '1999', 'element': 'Emissions Share (...",2021-11-07T05:29:57.595-06:00
4,agri-environmental-indicators-emissions-by-sector,3f4119c9755d2fddac4126a472f149131a6bace1,"{'year': '2012', 'element': 'Emissions Share (...",2021-11-07T05:29:57.595-06:00


In [7]:
# with open(json_file, "r") as df:
#     aggschema = load(df)
    
#     pretty_json2 = json.dumps(aggschema, indent=3)
#     print(pretty_json2)

### IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

In [8]:
fields_df = entire_file_df.loc[:, "fields"]
fields_df.head()

0    {'year': '2014', 'element': 'Emissions (F-gase...
1    {'year': '2018', 'element': 'Emissions (F-gase...
2    {'year': '1991', 'element': 'Emissions Share (...
3    {'year': '1999', 'element': 'Emissions Share (...
4    {'year': '2012', 'element': 'Emissions Share (...
Name: fields, dtype: object

In [14]:
ent_json = pd.json_normalize(fields_df)
ent_json

,year,element,unit,area,item,value
0,2014,Emissions (F-gases),kilotonnes,Mongolia,Waste - food systems,0.0000
1,2018,Emissions (F-gases),kilotonnes,Mongolia,Waste - food systems,0.0000
2,1991,Emissions Share (CO2eq) (AR5),%,Mongolia,Food Household Consumption,0.9332
3,1999,Emissions Share (CO2eq) (AR5),%,Mongolia,Food Household Consumption,0.3126
4,2012,Emissions Share (CO2eq) (AR5),%,Mongolia,Food Household Consumption,0.6152
...,...,...,...,...,...,...
1683943,2000,Emissions (N2O),kilotonnes,South America,Waste - food systems,9.9872
1683944,2011,Emissions (N2O),kilotonnes,South America,Waste - food systems,13.0647
1683945,2016,Emissions (N2O),kilotonnes,South America,Waste - food systems,12.9916
1683946,1992,Emissions (F-gases),kilotonnes,South America,Waste - food systems,0.0000


In [13]:
cntry_all = ent_json["area"].unique()
cntry_all_df = pd.DataFrame(cntry_all)
cntry_all_df

,0
0,Mongolia
1,Montenegro
2,Mexico
3,Micronesia (Federated States of)
4,Mauritius
...,...
270,Cook Islands
271,Ghana
272,Gibraltar
273,Greece


In [10]:

counrty_list = ['United States', 'Mexico', 'Brazil', 'Columbia', 'Venezuela','Chile','Peru','India','Bangladesh','Sri Lanka',
                'Myanmar', 'Pakistan', 'Russia', 'Ukraine', 'Kazakhstan', 'Azerbaijan', 'Armenia', 'Afghanistan', 'Uzbekistan',
                'South Africa', 'Nigeria', 'Ghana', 'Morocco', 'Egypt', 'Kenya', 'China', 'Japan', 'Hong Kong', 'Singapore', 
                'Thailand', 'Malaysia', 'Korea', 'Viet Nam', 'United Kingdom', 'France', 'Germany', 'Norway', 'Italy', 
                'Spain', 'Portugal', 'Sweden', 'Turkey', 'Greece', 'Saudi Arabia', 'Bahrain', 'Syrian Arab Republic', 
                'Iraq', 'Qatar', 'United Arab Emirates', 'Kuwait']

In [11]:
# new_fields_df = pd.DataFrame.from_dict(fields_df)
# new_fields_df
Agri_emis_cntry = ent_json.loc[ent_json["area"].isin(counrty_list)]
Agri_emis_cntry

,year,element,unit,area,item,value
697,2000,Emissions Share (CO2),%,Mexico,Energy - food systems,12.4978
698,2009,Emissions Share (CO2),%,Mexico,Energy - food systems,12.8555
699,2013,Emissions Share (CO2),%,Mexico,Energy - food systems,11.8992
700,2016,Emissions Share (CO2),%,Mexico,Energy - food systems,12.5585
701,2018,Emissions Share (CO2),%,Mexico,Energy - food systems,13.1745
...,...,...,...,...,...,...
1680933,1991,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4648.9348
1680934,1996,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4678.1061
1680935,1997,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4646.0924
1680936,2016,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,1923.3412


In [23]:
Agri_emis_cntry = Agri_emis_cntry.dropna(how='any')
Agri_emis_cntry.reset_index(drop = True, inplace = True)
Agri_emis_cntry

,year,element,unit,area,item,value
0,2000,Emissions Share (CO2),%,Mexico,Energy - food systems,12.4978
1,2009,Emissions Share (CO2),%,Mexico,Energy - food systems,12.8555
2,2013,Emissions Share (CO2),%,Mexico,Energy - food systems,11.8992
3,2016,Emissions Share (CO2),%,Mexico,Energy - food systems,12.5585
4,2018,Emissions Share (CO2),%,Mexico,Energy - food systems,13.1745
...,...,...,...,...,...,...
302723,1991,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4648.9348
302724,1996,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4678.1061
302725,1997,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,4646.0924
302726,2016,Emissions (CO2eq) (AR5),kilotonnes,Sri Lanka,LULUCF,1923.3412


In [20]:
Agri_emis_sector = Agri_emis_cntry['item'].value_counts()
Agri_emis_sector_df = pd.DataFrame(Agri_emis_sector)
Agri_emis_sector_df
# Agri_emis_sector_df

,item
Land Use change,12593
Food Transport,12593
IPCC Agriculture,12593
All sectors without LULUCF,12593
Waste - food systems,12593
Energy,12593
LULUCF,12593
Farm-gate emissions,12593
All sectors with LULUCF,12593
Food Processing,12593


In [21]:
Agri_emis_sector_df.to_csv("Agri_emis_by_Sector.csv", index= True, header = True)

In [22]:
Agri_emis_gases = Agri_emis_cntry['element'].value_counts()
Agri_emis_gases_df = pd.DataFrame(Agri_emis_gases)
Agri_emis_gases_df

,element
Emissions Share (CO2eq) (AR5),32239
Emissions (CO2eq) (AR5),32239
Emissions (F-gases),31650
Emissions (N2O),31139
Emissions Share (N2O),31139
Emissions Share (CO2),29586
Emissions (CO2),29586
Emissions Share (CH4),29098
Emissions (CH4),29098
Emissions Share (F-gases),26954


### Since Emission Share has a unit of % it is thought that 2 separate dataframes would be beneficial for loading into Database for Query purposes

In [24]:
Agri_em_percent = Agri_emis_cntry.loc[(Agri_emis_cntry["element"] == "Emissions Share (CO2eq) (AR5)") | 
                                      (Agri_emis_cntry["element"] == "Emissions Share (N2O)") |
                                        (Agri_emis_cntry["element"] == "Emissions Share (CO2)") |
                                      (Agri_emis_cntry["element"] == "Emissions Share (CH4)"), :]
Agri_em_percent
                                   
                                      

,year,element,unit,area,item,value
0,2000,Emissions Share (CO2),%,Mexico,Energy - food systems,12.4978
1,2009,Emissions Share (CO2),%,Mexico,Energy - food systems,12.8555
2,2013,Emissions Share (CO2),%,Mexico,Energy - food systems,11.8992
3,2016,Emissions Share (CO2),%,Mexico,Energy - food systems,12.5585
4,2018,Emissions Share (CO2),%,Mexico,Energy - food systems,13.1745
...,...,...,...,...,...,...
302718,2001,Emissions Share (N2O),%,Sri Lanka,LULUCF,0.0000
302719,2004,Emissions Share (N2O),%,Sri Lanka,LULUCF,0.0259
302720,2012,Emissions Share (N2O),%,Sri Lanka,LULUCF,0.0614
302721,2016,Emissions Share (N2O),%,Sri Lanka,LULUCF,0.0475
